How Big should the redistribution time differences be? 

Also every redistribution causes the file to increase the size a little bit :
the opening and closing of collection also has an effect on how much the size increases.

on vibes alone i think the difference is never more than 0.3s always -ve for some reason

In [3]:
from importlib import reload
import ananke.models.collection
from ananke.models.collection import Collection
reload(ananke.models.collection)
import ananke.models.collection
from ananke.configurations.collection import MergeConfiguration
from ananke.schemas.event import RecordType
from ananke.configurations.events import EventRedistributionMode
from ananke.configurations.collection import HDF5StorageConfiguration
import Trigger_Parallel_Working as tc
reload(tc)
import Trigger_Parallel_Working as tc
import cProfile
import pstats
import matplotlib.pyplot as plt
import os
import logging

# Configure logging to display DEBUG messages
logging.basicConfig(level=logging.INFO)

In [2]:
path='data/LargeStarting/0.h5'
config=HDF5StorageConfiguration(data_path=path, read_only=False)
C1=Collection(config)

In [ ]:
RecordType.REALISTIC_TRACK

In [ ]:
with C1:
    hits= C1.storage.get_hits().df
    records=C1.storage.get_records().df

In [8]:
type(configuration)

ananke.configurations.collection.MergeConfiguration

In [4]:
configuration = MergeConfiguration.parse_obj(
    {
        'in_collections': [
            {
                'type': 'hdf5',
                'data_path':'data/LargeTrack/20records/1.h5',
                'read_only':'False',
            },
            {
                'type': 'hdf5',
                'data_path': 'data/LargeElectrical/100000s/1.h5',
                'read_only':'False',
            },
            {
                'type': 'hdf5',
                'data_path': 'data/LargeBio/100000s/1.h5',
                'read_only':'False',
            },
        ],
        'out_collection': {
                'type': 'hdf5',
                'data_path': 'data/LargeTMerge/1.h5',
                'read_only':'False',
                
        },
        'content': [
            {
                'primary_type': RecordType.REALISTIC_TRACK.value,
                'secondary_types': [RecordType.ELECTRICAL.value,RecordType.BIOLUMINESCENCE],
                'number_of_records': 20,
                'interval': {
                    'start': 0,
                    'end': 1000000,
                }
            },
        ],
           
     }
)

#print(configuration)
#C2=Collection.from_merge(configuration)

INFO:root:Starting to merge collections with config.
INFO:root:Starting to create joined temporary collection.
100it [00:04, 23.21it/s]              
INFO:Collection:Starting to append collection.
100%|██████████| 100/100 [00:23<00:00,  4.27it/s]
INFO:Collection:Finished to append collection.
INFO:Collection:Starting to append collection.
100it [00:19,  5.23it/s]              
INFO:Collection:Finished to append collection.
INFO:root:Finished creating joined temporary collection.
INFO:root:Starting to create 20 RecordType.REALISTIC_TRACK records
INFO:root:Secondary types: 20, 21
100%|██████████| 20/20 [01:06<00:00,  3.32s/it]
INFO:root:Finished to create 20 RecordType.REALISTIC_TRACK records
INFO:root:Finished to merge collections with config.


In [6]:
type(configuration.redistribution)

NoneType

In [ ]:
with C2:
    Mhits= C2.storage.get_hits().df
    Mrecords=C2.storage.get_records().df

In [ ]:
display(hits.drop_duplicates(subset=['record_id','type']).head())
display(Mhits.drop_duplicates(subset=['record_id','type']).head())
display(Mhits1.drop_duplicates(subset=['record_id','type']).head())

In [ ]:
Mhits.shape[0],hits.shape[0]

In [ ]:
with C2:
    C2.redistribute(redistribution_configuration=configuration.redistribution)

In [ ]:
with C2:
    Mhits1= C2.storage.get_hits().df
    Mrecords1=C2.storage.get_records().df

In [ ]:
x=Mhits['time']-Mhits1['time']
x=x.round(6)
x_unique = x.drop_duplicates()
display(x_unique)

In [ ]:

time_diff = Mhits['time'] - Mhits1['time']
time_diff=time_diff.round(6)

# Add 'type' from Mhits to the time_diff series for grouping
time_diff_with_type = pd.DataFrame({'type': Mhits['type'], 'time_diff': time_diff})

# Group by 'type' and get unique time differences for each 'type'
unique_time_diffs_by_type = time_diff_with_type.groupby('type')['time_diff'].apply(lambda x: x.drop_duplicates())

# Display the results
display(unique_time_diffs_by_type)

In [ ]:
for i in range(10):
    with C2:
        configuration.redistribution.seed=i
        C2.redistribute(redistribution_configuration=configuration.redistribution)

Closing remaining open files:data/LargeBio/10000.h5...done/raven/u/arego/olympus/lib/python3.10/site-packages/ananke/configurations/../../_tmp_3ee34659-f126-476c-abd0-43de4539eb9cdata.h5...donedata/LargeElectrical/10000.h5...donedata/LargeElectrical/10000.h5...donedata/LargeBio/10000.h5...donedata/LargeMerge/0c.h5...done


In [2]:
import matplotlib.pyplot as plt
import numpy as np

def plot(hits):
    """
    Generate and display a histogram plot of 'time' for different 'type' values 
    within a specific 'record_id' in the given DataFrame.
    """
    # Select the first unique record_id
    record = hits['record_id'].drop_duplicates().iloc[0]
    
    # Get all unique types
    types = hits['type'].drop_duplicates()
    
    # Filter hits for the selected record_id
    hit = hits[hits['record_id'] == record]
    
    # Create a figure
    fig, ax = plt.subplots(figsize=(8, 5))
    
    interval = np.arange(0, 10000, 100)

    # Iterate through each unique type and plot its histogram
    for t in types:
        y, x = np.histogram(hit[hit['type'] == t]['time'], bins=interval)
        x = (x[:-1] + x[1:]) / 2  # Convert bin edges to bin centers
        ax.plot(x, y, label=f'Type {t}')

    # Labeling and legend
    ax.set_xlabel("Time")
    ax.set_ylabel("Frequency")
    ax.set_title(f"Histogram for Record ID {record}")
    ax.legend()

    # Show the plot each time the function is called
    plt.show()

# Example usage: